In [1]:
# !pip install ruclip==0.0.2

In [2]:
!nvidia-smi

Mon Sep 19 09:10:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:09:00.0 Off |                  N/A |
| 27%   34C    P8    15W / 250W |     10MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:42:00.0 Off |                  N/A |
| 67%   

In [3]:
import pandas as pd
import numpy as np
import torch
import os
import ruclip

device = "cuda" if torch.cuda.is_available() else "cpu"
# name_ruclip = 'ruclip-vit-base-patch16-224' #first clip metrica=0.600590
name_ruclip = "ruclip-vit-large-patch14-336"
clip, processor = ruclip.load(name_ruclip, device=device)
predictor = ruclip.Predictor(clip, processor, device, bs=1)

/usr/local/lib/python3.6/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 748k/748k [00:00<00:00, 1.35MB/s]
Downloading: 100%|██████████| 349/349 [00:00<00:00, 159kB/s]
Downloading: 100%|██████████| 1.71G/1.71G [08:35<00:00, 3.32MB/s] 


In [4]:
#считываем датафрейм, добавляем столбик для эмбеддингов
test_images_path = "/home/docker_current/datasets/train"
df_test = pd.read_csv("/home/docker_current/datasets/test.csv")
df_test['text_features'] = None
df_test['object_img'] = None

# считываем тестовые изображения, делаем словарь с эмбеддингами
test_images = [i.split('.png')[0] for i in os.listdir("/home/docker_current/datasets/test")]
test_embed = {i:None for i in test_images}

df_test.head(3)

,id,description,text_features,object_img
0,486,Фотография. Елизавета Алексеевна Юманова. ПКМ...,None,None
1,813,Фотография. Заседание комитета комсомола мотор...,None,None
2,2980,"Фотография. День ""Саланга"". ПОКМ-18530/638 фо...",None,None


In [5]:
from PIL import Image

def get_image_features(key):
    sample_image_path = "/home/docker_current/datasets/test/" + str(key) + ".png"
    image = Image.open(sample_image_path)
    
    with torch.no_grad():
        image_features = predictor.get_image_latents([image]).cpu().detach().numpy()
    return image_features

def get_text_features(df, index_text):
    sample_text = [df['description'][index_text]]
    text_features = predictor.get_text_latents(sample_text).cpu().detach().numpy()
    return text_features 

cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
def get_similarity(image_emb, text_emb):
    sim =cos(torch.Tensor(image_emb), 
                        torch.Tensor(text_emb))
    return sim

In [6]:
%%time
for ind_text in range(len(df_test)):
    df_test['text_features'][ind_text] = get_text_features(df_test, ind_text)

/home/docker_current/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


CPU times: user 46.2 s, sys: 63.2 ms, total: 46.2 s
Wall time: 46.3 s


In [7]:
for name_image in test_embed.keys():
    test_embed[name_image] = get_image_features(name_image)

1it [00:00, 33.18it/s]
1it [00:00, 42.19it/s]
1it [00:00, 53.97it/s]
1it [00:00, 54.64it/s]
1it [00:00, 45.77it/s]
1it [00:00, 46.62it/s]
1it [00:00, 47.46it/s]
1it [00:00, 43.49it/s]
1it [00:00, 46.46it/s]
1it [00:00, 60.58it/s]
1it [00:00, 28.80it/s]
1it [00:00, 22.56it/s]
1it [00:00, 19.24it/s]
1it [00:00, 33.76it/s]
1it [00:00, 41.04it/s]
1it [00:00, 50.93it/s]
1it [00:00, 53.58it/s]
1it [00:00, 37.41it/s]
1it [00:00, 49.98it/s]
1it [00:00, 47.64it/s]
1it [00:00, 38.22it/s]
1it [00:00, 39.37it/s]
1it [00:00, 40.11it/s]
1it [00:00, 33.56it/s]
1it [00:00, 50.27it/s]
1it [00:00, 46.96it/s]
1it [00:00, 44.52it/s]
1it [00:00, 52.26it/s]
1it [00:00, 46.05it/s]
1it [00:00, 55.73it/s]
1it [00:00, 49.97it/s]
1it [00:00, 42.16it/s]
1it [00:00, 41.57it/s]
1it [00:00, 23.90it/s]
1it [00:00, 34.05it/s]
1it [00:00, 48.30it/s]
1it [00:00, 41.58it/s]
1it [00:00, 40.53it/s]
1it [00:00, 55.53it/s]
1it [00:00, 51.92it/s]
1it [00:00, 49.57it/s]
1it [00:00, 47.10it/s]
1it [00:00, 50.14it/s]
1it [00:00,

In [8]:
from copy import deepcopy
copy_test_embed = deepcopy(test_embed)

In [9]:
preds = []

# ind_text = 0
for ind_text in range(len(df_test)):

    sims = []
    for image_name in copy_test_embed.keys():
        sim = get_similarity(copy_test_embed[image_name], 
                                    df_test['text_features'][ind_text])
        sims.append(sim)

    sims = np.array(sims)
    ind_max = np.argmax(sims)
    match_image = list(copy_test_embed.keys())[ind_max]
    preds.append(match_image)
    del copy_test_embed[match_image]

In [10]:
submit = pd.read_csv('/home/docker_current/py_files/sample_solution.csv')
submit['object_img'] = preds
submit['object_img'] = submit['object_img'].astype(np.int64)
submit.to_csv('ruclip-vit-large-patch14-336.csv', index=False)

In [48]:
# ind = 39
# text_features = get_text_features(df_test, ind)
# image_feat = get_image_features(list(test_embed.keys())[ind])

# get_similarity(text_features, image_feat)